In [1]:
import json
import logging
from re import sub
from multiprocessing import cpu_count

import numpy as np
import pandas as pd 

import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import nltk
from nltk.corpus import wordnet as wn

from gensim.models import Word2Vec

# pip install pymed

# nltk and pywsd # Libraries used 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer

import re, math
from re import sub

In [2]:
# https://towardsdatascience.com/how-to-compute-sentence-similarity-using-bert-and-word2vec-ab0663a5d64
# https://www.machinelearningplus.com/nlp/cosine-similarity/
# https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
# https://github.com/nlptown/nlp-notebooks/blob/master/Simple%20Sentence%20Similarity.ipynb    

In [3]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [4]:
len(model.vocab)

3000000

In [5]:
model.wv['stupefy']

<ipython-input-5-88cf1d6c9850>:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  model.wv['stupefy']


array([ 3.69140625e-01,  5.29785156e-02,  7.81250000e-02,  7.81250000e-02,
       -2.45117188e-01,  2.91015625e-01,  2.06054688e-01,  1.02539062e-02,
       -1.65039062e-01,  1.66015625e-01, -2.79296875e-01, -1.86523438e-01,
       -2.85644531e-02,  4.02343750e-01, -2.30468750e-01,  1.28906250e-01,
       -2.46582031e-02,  3.00781250e-01,  2.45117188e-01, -1.89453125e-01,
        1.21093750e-01, -7.42187500e-02,  1.74804688e-01, -4.95605469e-02,
        5.76171875e-02, -1.13769531e-01, -9.86328125e-02,  1.79687500e-01,
       -4.98046875e-02,  1.12304688e-01,  7.37304688e-02,  1.07910156e-01,
       -1.19628906e-01, -2.44140625e-02, -7.50732422e-03,  1.09375000e-01,
        2.11181641e-02, -4.44335938e-02,  2.03125000e-01,  5.78613281e-02,
        5.07812500e-02,  7.47070312e-02,  2.89062500e-01, -1.68945312e-01,
        1.42578125e-01, -1.16210938e-01,  4.88281250e-02,  9.42382812e-02,
        4.78515625e-02,  1.21093750e-01, -4.25781250e-01, -5.17578125e-02,
        1.04492188e-01, -

In [6]:
# Takes a paragraph and returns the signature.

#paragraph = str(obj['paragraph'])
def para_signature(paragraph, word):
    sent_sig = [] # Stores sentence signature with POS

    # 1. Tokenize and remove punctuation
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(paragraph)
    
    # For POS Tagging
    def find_pos(new_words):
        pt = nltk.pos_tag(new_words)
        pos1 = ""
        for i in pt:
            if i[0]== word:
                pos1 = (i[1])
            else:
                continue
        return pos1

    def add_pos(pos_value):
        # make a dictionary of possible tags (in progress)
        pos_tag = {'verb':['VB','VBD','VBG','VBN','VBP','VBZ'], 
                   'noun': ['NN','NNS','NNP'], 
                   'adjective': ['JJ','JJR','JJS'],
                   'adverb':['RB', 'RBR', 'RBS']} # "masculine noun",

        for ele in pos_tag:
            for i in pos_tag[ele]:
                if i == pos:
                    return ele
                else:
                    continue

    

    # 2. POS Tagging
    pos = find_pos(new_words)
    pos_to_list = [add_pos(pos)]

    # 3. Convert all to lower case
    words = [word.lower() for word in new_words]

    # 4. Filter out stop words
    stop_words = stopwords.words('english')
    words_rem = [w for w in words if not w in stop_words]
    #words_rem= pos_to_list + words_rem   #add POS label to the list
    #sent_sig.append(words_rem)
    
    # 4. Lemmatize the words
    lmtzr = WordNetLemmatizer()
    lemmatized = [lmtzr.lemmatize(word) for word in words_rem]
    lemmatized= pos_to_list + lemmatized   #add POS label to the list
    sent_sig = lemmatized

    
    return sent_sig

#para_sig = para_signature(paragraph,word)
#print(para_sig)

In [7]:
# Takes word meanings and returns its signature with POS.

def meaning_signatures(dict_data,lu_word):
    signatures = [] # Stores meaning signatures with POS
    tokenizer = nltk.RegexpTokenizer(r"\w+")

    for i in range(len(dict_data)):

        # 1. Tokenize and remove punctuation
        dict_meaning= tokenizer.tokenize(dict_data[i].get("meaning"))

        # 2. Convert all to lower case
        word_set = [word.lower() for word in dict_meaning]

        # 3. Filter out stop words
        stop_words = stopwords.words('english')
        signature = [w for w in word_set if not w in stop_words]

        # 4. Lemmatize the words
        lmtzr = WordNetLemmatizer()
        lemmatized = [lmtzr.lemmatize(word) for word in signature]
        lemmatized= [dict_data[i].get("fl")] + lemmatized #add POS label to the list
        #signatures.append(lemmatized)
        
        # 6. Remove the look up word if present in meaning
        lemmatized[:] = [x for x in lemmatized if lu_word not in x]
        # Add stemmed meanings to a list
        signatures.append(lemmatized)
        
        
    return (signatures)    

#meaning_sig = meaning_signatures(dict_data, word)
#meaning_sig

In [8]:
def extract_ids(dicdata):
    ids = []
    for i in dict_data:
        mea_id = i['id']
        ids.append(mea_id)
    return ids

def extract_meanings(dicdata):
    meaning_list = []
    for i in dict_data:
        meaning = i['meaning'] 
        meaning_list.append(meaning)
    return meaning_list

def extract_pos_meanings(dicdata):
    pos_meaning = []
    for i in dict_data:
        pos = i['fl']
        meaning = i['meaning'] 
        pos_mea = pos + ' '+ meaning # POS + meaning
        pos_meaning.append(pos_mea)
    return pos_meaning

def extract_isteacher(dicdata):
    isteacher = []
    for i in dict_data:
        is_tea = i['isTeacher']
        isteacher.append(is_tea)
    return isteacher

In [9]:
# 1. Input
# JSON file 
# Read file
jsonfile= open('Json_wsd/request29.json','r',encoding='utf8')
data= jsonfile.read()

# Parse and Extract word, paragraph, pos and meanings
obj = json.loads(data)

word = str(obj['word'])
print("\nLookup word: ",word)

paragraph = str(obj['paragraph'])
print("\nParagraph: ",paragraph)

dict_data = obj['dictionaryData']
print(dict_data)
ids = extract_ids(dict_data)
meanings = extract_meanings(dict_data)
print(meanings)
#pos_meanings = extract_pos_meanings(dict_data)
#print("\n",pos_meanings)
isteacher = extract_isteacher(dict_data)



Lookup word:  swagger

Paragraph:  Mild-mannered, he walked with more hunch than swagger. In class, he preferred listening to speaking. Patrick never bullied anybody. He never cussed anybody out. He appeared to abide by a self-imposed code: Keep to yourself, don’t mess around, don’t get involved with other people’s trouble. But he was willing to break his own code for a cause: Once, he leapt between two girls to stop a fight, and, in the process, got slammed to the ground.
[{'id': 2743, 'word_id': 861, 'fl': 'noun', 'meaning': 'a way of walking or behaving that shows you have a lot of confidence', 'count': 2, 'isTeacher': 1, 'isUser': 0}, {'id': 2744, 'word_id': 861, 'fl': 'verb', 'meaning': 'to walk in a very confident way : to walk with a swagger', 'count': 0, 'isTeacher': None, 'isUser': 0}, {'id': 5962, 'word_id': 861, 'fl': 'verb', 'meaning': 'to conduct oneself in an arrogant or superciliously pompous manner; especially : to walk with an air of overbearing self-confidence', 'cou

In [10]:
para_sig = para_signature(paragraph,word)
meaning_sig = meaning_signatures(dict_data, word)

vocab_w2v = list(model.wv.vocab) # list of all vocab
#vocab_w2v_wiki = list(word_vectors.wv.vocab) # list of all vocab
para_sig_r = [w for w in para_sig if w in vocab_w2v]

meaning_sig_r =[]
for i in meaning_sig:
    m_sig_r = [w for w in i if w in vocab_w2v]
    meaning_sig_r.append(m_sig_r)

# Find cosine similarity    
cos_sim = []
for i in meaning_sig_r:
    similarity =model.n_similarity(para_sig_r, i)
    cos_sim.append(similarity)
print(cos_sim)

zipped = list(zip(ids, meanings, cos_sim, isteacher))
features = sorted(zipped, key = lambda x: x[2], reverse = True)


ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
ranked_meanings

<ipython-input-10-0cbaf3bd97f7>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab_w2v = list(model.wv.vocab) # list of all vocab


[0.70725626, 0.5990579, 0.5972239, 0.6842869, 0.33560646, 0.5751412]


,id,meaning,score,isTeacher
0,2743,a way of walking or behaving that shows you ha...,0.707256,1.0
1,5963,an arrogantly self-confident way of walking : ...,0.684287,0.0
2,2744,to walk in a very confident way : to walk with...,0.599058,0.0
3,5962,to conduct oneself in an arrogant or supercili...,0.597224,0.0
4,5965,a short light stick usually covered with leath...,0.575141,0.0
5,5964,marked by elegance or showiness : posh,0.335606,0.0


In [11]:
# Load Data and store

#request1train1_revised.json
jsonfile= open('data_600.json','r',encoding='utf8') # removed word aimless
data= jsonfile.read() 

# Parse and Extract word, paragraph, pos and meanings
obj = json.loads(data)

data = obj['data'] #[:10]
print("Total words: ",len(data)) # Total # of words

vocab_w2v = list(model.wv.vocab) # list of all vocab

count=0
# Store accuracy and MRR in blank lists
acc_score=[]
mrr_score=[]

# Parse data and find best meaning
for each_word_data in data:
    word = each_word_data['word']
    paragraph = each_word_data['paragraph']
    dict_data = each_word_data['dictionaryData']
    for i in dict_data:
        ids = extract_ids(dict_data)
        meanings = extract_meanings(dict_data)
        pos_meanings = extract_pos_meanings(dict_data)
        isteacher = extract_isteacher(dict_data)
        
        
    # Get signatures    
    para_sig = para_signature(paragraph,word)
    meaning_sig = meaning_signatures(dict_data, word)
    
    # Add tokens of meaning if meaning list is less than 2 words
    meaning_sig1 =[]
    for i,sig in enumerate(meaning_sig):
        if len(sig)<2:
            tokenizer = nltk.RegexpTokenizer(r"\w+")
            dict_meaning= tokenizer.tokenize(dict_data[1].get("meaning"))
            sig = sig+ dict_meaning
            meaning_sig1.append(sig)
        else:
            meaning_sig1.append(sig)  

    
    # Remove words not in Vocab
    #vocab_w2v = list(model.wv.vocab) # list of all vocab
    para_sig_r = [w for w in para_sig if w in vocab_w2v]

    meaning_sig_r =[]
    for i in meaning_sig1:
        m_sig_r = [w for w in i if w in vocab_w2v]
        meaning_sig_r.append(m_sig_r)

    # Compute Cosine Similarity Measure between the query and the documents.   
    cos_sim = []
    for i in meaning_sig_r:
        similarity =model.n_similarity(para_sig_r, i)
        cos_sim.append(similarity)
    
    
    # Output the document similarity results
    zipped = list(zip(ids, meanings, cos_sim, isteacher))
    features = sorted(zipped, key = lambda x: x[2], reverse = True)

    print("\n\nLookup word: ",word)
    ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
    ranked_meanings['isTeacher'] = ranked_meanings['isTeacher'].fillna(0)
    ranked_meanings['pred_rank'] = ranked_meanings['score'].rank(pct=False, ascending= False)
    print(ranked_meanings)
    
     
    count +=1
    print(count)
    
    # Find accuracy
    if ranked_meanings.iloc[0]['isTeacher'] ==1:
        acc_score.append(1)
    else:
        acc_score.append(0)
        
    # Find MRR score
    istea_index = ranked_meanings[ranked_meanings['isTeacher']==1].index[0]+1
    pred_rank =1/istea_index
    mrr_score.append(pred_rank)

# Return Accuracy and MRR 
#print(acc_score)
avg = sum(acc_score)/len(acc_score)
print("The accuracy score is ", round(avg,4))

mrr = np.sum(mrr_score)/ len(mrr_score)
print("The Mean Reciprocal Rank is ", round(mrr,4)) 

Total words:  600


<ipython-input-11-c1d568cbb0d7>:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocab_w2v = list(model.wv.vocab) # list of all vocab




Lookup word:  circumvent 
     id                                            meaning     score  \
0  3230  to avoid being stopped by (something, such as ...  0.575862   

   isTeacher  pred_rank  
0          1        1.0  
1


Lookup word:  addict
     id                                            meaning     score  \
0  3401  a person who is not able to stop taking drugs ...  0.591445   
1  5885                    to cause addiction in (someone)  0.454675   
2  5884  one exhibiting a compulsive, chronic, physiolo...  0.440599   

   isTeacher  pred_rank  
0        0.0        1.0  
1        0.0        2.0  
2        1.0        3.0  
2


Lookup word:  frustrated
     id                                            meaning     score  \
0  5887                   to balk or defeat in an endeavor  0.478687   
1  5886  feeling, showing, or characterized by frustrat...  0.471775   

   isTeacher  pred_rank  
0        0.0        1.0  
1        1.0        2.0  
3


Lookup word:  manslaughter
  



Lookup word:  copious
     id                         meaning     score  isTeacher  pred_rank
0  5933   yielding something abundantly  0.499475        0.0        1.0
1  2558  very large in amount or number  0.409221        1.0        2.0
21


Lookup word:  zeal
     id                                            meaning     score  \
0  3319  a strong feeling of interest and enthusiasm th...  0.699908   
1  5935                                   great enthusiasm  0.551507   
2  5934  eagerness and ardent interest in pursuit of so...  0.536322   
3  3320                     of or relating to missionaries  0.395260   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        0.0        4.0  
22


Lookup word:  convictions
     id                                            meaning     score  \
0  2576  the ability to do something that you know is d...  0.639379   
1  5936  the act or process of finding a person guilty ...  0.617216  



Lookup word:  ghetto
     id                                            meaning     score  \
0  2751  a part of a city in which members of a particu...  0.655691   
1  5982  a quarter of a city in which Jews were formerl...  0.571947   
2  5983  of, relating to, being, or characteristic of a...  0.548505   
3  2752                                           boom box  0.401435   
4  5984                                          ghettoize  0.382793   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        0.0        4.0  
4        0.0        5.0  
42


Lookup word:  decay
     id                                            meaning     score  \
0  5988  to go slowly from a bad condition to a worse c...  0.633850   
1  5986  rot; specifically : aerobic decomposition of p...  0.415468   
2  5989                A state of rotting or decomposition  0.401612   
3  5987  a radioactive nuclear transformation governed ...  0.401416   
4  

...



Lookup word:  trigger
     id                                            meaning     score  \
0  2676  to cause (something, such as an alarm) to star...  0.583738   
1  2678  a trigger on a gun that needs very little pres...  0.546210   
2  2680  to hold onto and move (someone or something) i...  0.545482   
3  2675     a lever on a gun that you pull to fire the gun  0.450830   
4  2679                         becoming angry very easily  0.448227   
5  2677                                eager to fire a gun  0.392700   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        0.0        4.0  
4        0.0        5.0  
5        0.0        6.0  
494


Lookup word:  repudiated 
     id                                         meaning     score  isTeacher  \
0  4736  to divorce or separate formally from (a woman)  0.383071          1   

   pred_rank  
0        1.0  
495


Lookup word:  inmates
     id                               



Lookup word:  embedded
     id                                            meaning     score  \
0  1469  to place or set (something) firmly in somethin...  0.608112   
1  6846  occurring as a grammatical constituent (such a...  0.456465   
2  6847         to enclose closely in or as if in a matrix  0.341986   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
515


Lookup word:  envisioned
     id                                            meaning     score  \
0  2341  to think of (something that you believe might ...  0.711516   
1  5819                              to picture to oneself  0.443551   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
516


Lookup word:  sealed
     id                                            meaning     score  \
0  6852    to confirm or make secure by or as if by a seal  0.626345   
1  6851                                      to hunt seals  0.550931   
2  6856      



Lookup word:  mercy
     id                                            meaning     score  \
0  6902  inclined to treat other people with kindness a...  0.561795   
1  6899  in a position or situation in which one can be...  0.555602   
2  6905                         to beg (someone) for mercy  0.534180   
3  6897  compassion or forbearance shown especially to ...  0.530248   
4  6906  without any protection from (someone or someth...  0.515202   
5  6904                          to treat (someone) kindly  0.494196   
6  6900             to treat with kindness and forgiveness  0.482651   
7  6903                    to treat (someone) very harshly  0.465224   
8  6901  a trip to help people (such as victims of a di...  0.420665   
9  6898                                         euthanasia  0.279436   

   isTeacher  pred_rank  
0        0.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        1.0        4.0  
4        0.0        5.0  
5        0.0        6.0  
6  



Lookup word:  sacred
     id                                            meaning     score  \
0  6947  one that is often unreasonably immune from cri...  0.491713   
1  6945  dedicated or set apart for the service or wors...  0.469942   
2  6950  a baboon (Papio hamadryas) that has a reddish-...  0.467895   
3  6949                                     magic mushroom  0.381734   
4  6948                                             peepul  0.361266   
5  6946                                   hamadryas baboon  0.270392   

   isTeacher  pred_rank  
0        1.0        1.0  
1        0.0        2.0  
2        0.0        3.0  
3        0.0        4.0  
4        0.0        5.0  
5        0.0        6.0  
567


Lookup word:  contraband
     id                                            meaning     score  \
0  4682  goods or merchandise whose importation, export...  0.443588   

   isTeacher  pred_rank  
0          1        1.0  
568


Lookup word:  threshold
     id                         



Lookup word:  alternative
      id                                            meaning     score  \
0   3481  something that can be chosen instead of someth...  0.731165   
1   6153  a proposition or situation offering a choice b...  0.696100   
2   6158  a U.S. federal income tax that was originally ...  0.509843   
3   6157  any of various systems of healing or treating ...  0.492171   
4   6159  music that is produced by performers who are o...  0.478270   
5   6160  pop music that has broad appeal but that is pr...  0.468205   
6   6155  usable power (such as heat or electricity) tha...  0.441782   
7   6161  alternative music that blends elements of conv...  0.425820   
8   6156  a fuel for internal combustion engines that is...  0.422918   
9   3480                    offering or expressing a choice  0.367977   
10  6154                                        alt-country  0.321413   

    isTeacher  pred_rank  
0         1.0        1.0  
1         0.0        2.0  
2         0.0 